In [13]:
import pandas as pd
import os
from IPython.display import display
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import EfficientNetB7,preprocess_input
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import GroupKFold

## CFG

In [14]:
class CFG:
    debug=True

    batch_size=8 if debug else 1024
    epochs=1 if debug else 20
    n_splits=4

    dataset_dir="../input/ranzcr-clip-catheter-line-classification/"
    target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']

In [15]:
train=pd.read_csv(CFG.dataset_dir+"train.csv")
gen_df=train.copy()
gen_df["StudyInstanceUID"]=gen_df["StudyInstanceUID"]+".jpg"

group_kfold=GroupKFold(n_splits=CFG.n_splits)
for train_idx,val_idx in group_kfold.split(gen_df,groups=gen_df["PatientID"].values):
    train_gen_df=gen_df.iloc[train_idx,:]
    val_gen_df=gen_df.iloc[val_idx,:]
    break



datagen=image.ImageDataGenerator(preprocessing_function=preprocess_input)

def get_generator(datagen,df):
    generator=datagen.flow_from_dataframe(
    directory=CFG.dataset_dir+"train", dataframe=df,
    x_col="StudyInstanceUID", y_col=CFG.target_cols, class_mode="raw",
    target_size=(256,256), batch_size=CFG.batch_size, seed=43
    )

    return generator

train_generator=get_generator(datagen,train_gen_df)
val_generator=get_generator(datagen,val_gen_df)

Found 22562 validated image filenames.
Found 7521 validated image filenames.


In [16]:
def check_generator_output():
    imgs,labels=next(train_generator)

    fig, axes=plt.subplots(2,4,figsize=(30,15))
    axes=axes.flatten()
    for img,label,ax in zip(imgs,labels,axes):
        ax.imshow(img)
        ax.set_title(label)
    plt.show()

In [17]:
efficientnet_b7=EfficientNetB7(include_top=False,weights="imagenet",pooling="avg")
efficientnet_b7.trainable=False

In [18]:
transfer_model=Sequential([
    efficientnet_b7,
    Dense(11,activation="sigmoid")
])
transfer_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb7 (Functional)  (None, 2560)              64097687  
_________________________________________________________________
dense_1 (Dense)              (None, 11)                28171     
Total params: 64,125,858
Trainable params: 28,171
Non-trainable params: 64,097,687
_________________________________________________________________


In [12]:
adam=Adam(learning_rate=1e-3)
transfer_model.compile(optimizer=adam,loss="binary_crossentropy",metrics=[keras.metrics.AUC(multi_label=True)])

lr_reducer = keras.callbacks.ReduceLROnPlateau(monitor="val_auc", patience=3, min_lr=1e-6, mode='max')

transfer_model.fit(x=train_generator,callbacks=[lr_reducer],max_queue_size=2,
    epochs=CFG.epochs,steps_per_epoch=len(train_gen_df)//CFG.batch_size,
    validation_data=val_generator,validation_steps=len(val_gen_df)//CFG.batch_size
)

KeyboardInterrupt: 